In [1]:
# Parameters
num_cores = 27
mem = 650000
config = "configs.config_Reddit"


In [2]:
### The steps of preprocessing:

# 1- cleaning
# 2- Unigram
#     2-1 - English
#     2-2 - Stop Words
#     2-3 - Stemming
# 3- Bigram
# 4- Tokenization
# 6- Lemmatization
# 7- Thresholding: 
#     7-1 min < word < % of corupus
#     7-2 min_words_per_docs
# 8- LDA

### The steps of preprocessing for tokens:

# 1- cleaning
# 2- Stemming

In [3]:
#!/usr/bin/env python
# coding: utf-8

if 'config' not in locals():
#     config = 'configs.config_SE_test_local'
    config = 'configs.config_Reddit_test_local'

In [4]:
### initialization ###

from _imports import *
# from _utils import *

print('config file is set to {}'.format(config))

util=importlib.import_module('_utils')
importlib.reload(util)

c=importlib.import_module(config)
importlib.reload(c)

get_ipython().run_line_magic('matplotlib', 'inline')

# param_setup(sys.argv[1:], c)
util.param_setup_ipython(globals(), c)

config file is set to configs.config_Reddit
num_cores is set to 27
mem is set to 650000


In [5]:
### text files loading ###


# function for dataframe checking ###

def df_checking(df, file_dict, df_name):
    count_dict = dict()
    def check_timestamp(timestamp):
        #     print(timestamp)
        try:
            datetime.fromtimestamp(int(timestamp))
        except ValueError as e:
            count_dict[str(timestamp)] = count_dict.get(str(timestamp), 0) + 1
        except TypeError as e:
            count_dict['None'] = count_dict.get('None', 0) + 1
        
    if file_dict['cols']['id'] not in df.columns:
        raise ValueError('{} doesn\'t have {} column!'.format(df_name, file_dict['cols']['id']))
    if file_dict['cols']['date'] not in df.columns:
        raise ValueError('{} doesn\'t have date column!'.format(df_name, file_dict['cols']['date']))
    if file_dict['cols']['category'] and file_dict['cols']['category'] not in df.columns:
        raise ValueError('{} doesn\'t have {} column!'.format(df_name, file_dict['cols']['category']))
    for text_col in file_dict['cols']['text']:
        if text_col not in df.columns:
            raise ValueError('{} doesn\'t have {} column!'.format(df_name, text_col))

    if file_dict['formats']['date'] == 'timestamp':
        count_dict = dict()
        df[file_dict['cols']['date']].apply(check_timestamp)
    else:
        for date in df[file_dict['cols']['date']]:
            try:
                pd.to_datetime(date, format=file_dict['formats']['date'])
            except ValueError as e:
                count_dict[str(date)] = count_dict.get(str(date), 0) + 1

    print('{} file has {} corrupt rows!'.format(df_name, count_dict))

    
dfs = dict()

for file in c.template['files'].keys():

    fname = list()

    for f in os.listdir(c.directory['files'] + file):
        if (f.find('.{}'.format(c.template['files'][file]['formats']['file'])) != -1 and f[0] != '.'):
            fname.append(f)

    fname=sorted(fname)
    pprint(fname)

### input file should be cleaned from \r using the bash script

    if len(fname)>0:
        dfs[file] = pd.read_csv(c.directory['files'] + file + '/' + fname[0],
                              sep=c.template['files'][file]['formats']['sep'])
        df_checking(dfs[file], c.template['files'][file], fname[0])

        for f in fname[1:]:
            df_temp = pd.read_csv(c.directory['files'] + file + '/' + f,
                                    sep=c.template['files'][file]['formats']['sep'],
                                    skip_blank_lines=True
                                    )
            df_checking(df_temp, c.template['files'][file], f)
            dfs[file] = pd.concat([dfs[file], df_temp], axis=0, sort=True)
    else:
        dfs[file]=pd.DataFrame() 
        

    pprint('Number of rows for {} = {}'.format(file, len(dfs[file])))
    util.display_df(dfs[file].head())

['all_RS_2014-01_all.csv',
 'all_RS_2014-02_all.csv',
 'all_RS_2014-03_all.csv',
 'all_RS_2014-04_all.csv',
 'all_RS_2014-05_all.csv',
 'all_RS_2014-06_all.csv',
 'all_RS_2014-07_all.csv',
 'all_RS_2014-08_all.csv',
 'all_RS_2014-09_all.csv',
 'all_RS_2014-10_all.csv',
 'all_RS_2014-11_all.csv',
 'all_RS_2014-12_all.csv',
 'all_RS_2015-01_all.csv',
 'all_RS_2015-02_all.csv',
 'all_RS_2015-03_all.csv',
 'all_RS_2015-04_all.csv',
 'all_RS_2015-05_all.csv',
 'all_RS_2015-06_all.csv',
 'all_RS_2015-07_all.csv',
 'all_RS_2015-08_all.csv',
 'all_RS_2015-09_all.csv',
 'all_RS_2015-10_all.csv',
 'all_RS_2015-11_all.csv',
 'all_RS_2015-12_all.csv',
 'all_RS_2016-01_all.csv',
 'all_RS_2016-02_all.csv',
 'all_RS_2016-03_all.csv',
 'all_RS_2016-04_all.csv',
 'all_RS_2016-05_all.csv',
 'all_RS_2016-06_all.csv',
 'all_RS_2016-07_all.csv',
 'all_RS_2016-08_all.csv',
 'all_RS_2016-09_all.csv',
 'all_RS_2016-10_all.csv',
 'all_RS_2016-11_all.csv',
 'all_RS_2016-12_all.csv',
 'all_RS_2017-01_all.csv',
 

,author,created_utc,id,num_comments,num_crossposts,permalink,score,selftext,subreddit,subreddit_id,subreddit_type,title
0,nathancrumpton,1390156402,1vlv7n,2,NaN,/r/dataisbeautiful/comments/1vlv7n/the_top_20_...,1,NaN,dataisbeautiful,t5_2tk95,NaN,"The top 20 Metro Areas in the United States, 1..."
1,[deleted],1390157365,1vlwln,1,NaN,/r/MachineLearning/comments/1vlwln/in_the_begi...,0,"Near the beginning of awareness, the Universal...",MachineLearning,t5_2r3gv,NaN,In the Beginning
2,wanderingstan,1390160408,1vm12a,4,NaN,/r/dataisbeautiful/comments/1vm12a/influence_m...,8,NaN,dataisbeautiful,t5_2tk95,NaN,(Influence mapping of) Programming Languages a...
3,poopballs,1390168955,1vmdrv,3,NaN,/r/dataisbeautiful/comments/1vmdrv/comparison_...,1,NaN,dataisbeautiful,t5_2tk95,NaN,Comparison Chart of different aircraft carrier...
4,Mister_Six,1390169441,1vmei2,1,NaN,/r/dataisbeautiful/comments/1vmei2/welfare_fra...,1,NaN,dataisbeautiful,t5_2tk95,NaN,Welfare fraud vs Tax evasion/avoidance estimat...


['all_RC_2014-01_all_.csv',
 'all_RC_2014-02_all_.csv',
 'all_RC_2014-03_all_.csv',
 'all_RC_2014-04_all_.csv',
 'all_RC_2014-05_all_.csv',
 'all_RC_2014-06_all_.csv',
 'all_RC_2014-07_all_.csv',
 'all_RC_2014-08_all_.csv',
 'all_RC_2014-09_all_.csv',
 'all_RC_2014-10_all_.csv',
 'all_RC_2014-11_all_.csv',
 'all_RC_2014-12_all_.csv',
 'all_RC_2015-01_all_.csv',
 'all_RC_2015-02_all_.csv',
 'all_RC_2015-03_all_.csv',
 'all_RC_2015-04_all_.csv',
 'all_RC_2015-05_all_.csv',
 'all_RC_2015-06_all_.csv',
 'all_RC_2015-07_all_.csv',
 'all_RC_2015-08_all_.csv',
 'all_RC_2015-09_all_.csv',
 'all_RC_2015-10_all_.csv',
 'all_RC_2015-11_all_.csv',
 'all_RC_2015-12_all_.csv',
 'all_RC_2016-01_all_.csv',
 'all_RC_2016-02_all_.csv',
 'all_RC_2016-03_all_.csv',
 'all_RC_2016-04_all_.csv',
 'all_RC_2016-05_all_.csv',
 'all_RC_2016-06_all_.csv',
 'all_RC_2016-07_all_.csv',
 'all_RC_2016-08_all_.csv',
 'all_RC_2016-09_all_.csv',
 'all_RC_2016-10_all_.csv',
 'all_RC_2016-11_all_.csv',
 'all_RC_2016-12_ala

,author,author_fullname,body,controversiality,created_utc,gilded,id,link_id,parent_id,permalink,removal_reason,score,subreddit,subreddit_id
0,Uncentury,NaN,As someone who lives and Memphis and sees FedE...,0,1389915441,0,cerg1q8,t3_1vck13,t3_1vck13,NaN,NaN,2,dataisbeautiful,t5_2tk95
1,kerbuffel,NaN,There's music playing in the background,0,1389915509,0,cerg2tw,t3_1vck13,t1_cer6gho,NaN,NaN,2,dataisbeautiful,t5_2tk95
2,SlackingAlready,NaN,"Interesting, but keep in mind that this sample...",0,1389915629,0,cerg4rl,t3_1veeqv,t3_1veeqv,NaN,NaN,250,dataisbeautiful,t5_2tk95
3,rand000m,NaN,"""the owner of this video has not made it avail...",0,1389916005,0,cergasd,t3_1vck13,t3_1vck13,NaN,NaN,2,dataisbeautiful,t5_2tk95
4,hockeyfan1133,NaN,Correlation is not causation.,0,1389916117,0,cergcle,t3_1ve29c,t1_cerfej0,NaN,NaN,14,dataisbeautiful,t5_2tk95


In [6]:
### tags files loading, cleaning, stemming & saving ###

if c.tags is not None:
    ps = PorterStemmer()
    tags = pd.read_csv(c.directory['tags'] + c.tags['name'])
    pprint('tags = {}'.format(len(tags)))
    util.display_df(tags.head())

    tags_imported = list(tags[c.tags['col_name']])
    tags_imported.extend(c.tags['extra'])

    tags_list = [tag.replace('-', ' ') for tag in tags_imported]
    
    stemmed_tags_list=[]
    for tags in tags_list:
        res=[]
        for tag_word in tags.split():
            res.append(ps.stem(tag_word))
        stemmed_tags_list.append(' '.join(res))
    stemmed_tags_list=list(set(stemmed_tags_list))
    stemmed_tags_list.sort(key=len, reverse=True)

    tags_dict = {}
    for tag_id, tag in enumerate(stemmed_tags_list):
        tags_dict[tag_id] = tag
    df_tags = pd.DataFrame(zip(tags_dict.keys(), tags_dict.values()), columns=['key', 'val'])
    df_tags.to_csv('{}{}_0_tags.csv'.format(c.directory['save'], c.project_name), index=False)

    pprint('tags = {}'.format(len(df_tags)))
    util.display_df(df_tags.head())

'tags = 449'


,Id,TagName,Count
0,1,definitions,23
1,2,machine-learning,4766
2,3,bigdata,349
3,5,data-mining,790
4,6,databases,58


'tags = 451'


,key,val
0,0,mini batch gradient descent
1,1,microsoft bot framework
2,2,evolutionari algorithm
3,3,social network analysi
4,4,natur languag process


In [7]:
### functions for time correction ###

def concat_cols(row):
    return ' '.join(map(str, row))

def concat_cols_df(df):
    if isinstance(df, pd.DataFrame):
        if len(df.columns) == 1:
            return df
        return df.apply(concat_cols, axis=1)
    elif isinstance(df, pd.Series):
        return df.apply(concat_cols)
    raise Failed('Input was not either dataframe or series!')

def str_timestamp_to_date(date):
    try:
        return datetime.fromtimestamp(int(date))
    except ValueError as e:
        util.count_sync(10)
        return default_date

def str_timestamp_to_date_df(df):
    return df.apply(str_timestamp_to_date)

def str_timestamp_to_date_df_dask(df):
    return df['date'].apply(str_timestamp_to_date)

def str_date_to_date(date):
    try:
        return pd.to_datetime(df_text['date'], format=c.template['files'][df_name]['formats']['date'])
    except ValueError as e:
        util.print_asynch(date)
        return default_date


def date_to_timestamp(date):
    try:
        return int(date.replace(tzinfo=timezone.utc).timestamp())
    except Exception as e:
        raise util.Failed('{}\n{} has type with {} value'.format(e, type(date), date))

In [8]:
### text concatenation + date correction + date filtering (min & max) & saving ###

dfs_text = pd.DataFrame()

for df_name in c.template['files']:
    # df_name='submissions'
    if(len(dfs[df_name])==0):
        continue
    pprint('files name = {}'.format(df_name))
    df = c.template['files'][df_name]['cols']
    cols = [df['id']]
    cols.append(df['date'])
    if df['category']:
        cols.append(df['category'])
    cols.extend(df['text'])
    df_text = dfs[df_name][cols]
    df_text = df_text.fillna(value='')
    df_text.reset_index(inplace=True)
    
#     util.init_parallel(False, c)
#     df_text['all_text'] = df_text[df['text']].parallel_apply(concat_cols, axis=1)
#     util.display_df(df_text[df['text']].head(30))
#     util.display_df(df_text[df['text']].tail())
    df_text['all_text'] = util.parallelize_df(df_text[df['text']], concat_cols_df, c)
#     print(df_text['all_text'])
    
    
    df_text.drop(df['text'], axis=1, inplace=True)
    df_text.rename(columns={
        df['id']: 'id',
        df['date']: 'date',
    }, inplace=True)
    if df['category']:
        df_text.rename(columns={df['category']: 'category'}, inplace=True)
    min_date = pd.to_datetime(c.min_date_val, format=c.date_format)
    max_date = pd.to_datetime(c.max_date_val, format=c.date_format)
    default_date = min_date - timedelta(days=10)

    
    if c.template['files'][df_name]['formats']['date'] == 'timestamp':
#         util.init_parallel(False, c)
#         df_text['date_converted'] = df_text['date'].apply(str_timestamp_to_date)
#         df_text['date_converted'] = util.parallelize_df(df_text['date'], str_timestamp_to_date_df, c)
        df_text['date_converted'] = util.parallelize_df_dask(df_text['date'], str_timestamp_to_date_df, c)    
#         df_text['date_converted'] = df_text['date'].apply(str_timestamp_to_date)    
        print('str_timestamp_to_date step is done!')
    else:
        df_text['date_converted'] = util.parallelize_df(df_text['date'], str_date_to_date, c)
#         df_text['date_converted'] = util.parallelize_df_dask(df_text['date'], str_date_to_date, c)        
        print('str_date_to_date step is done!')

        # df_text['date']=c.parallelize_df(c.copy.deepcopy(df_text['date_converted']), date_to_timestamp)
        # print(type(df_text['date_converted'][0]))
        df_text['date'] = df_text['date_converted'].apply(date_to_timestamp)
        print('int_date_to_timestamp step is done!')

    df_text = df_text[(df_text['date_converted'] <= max_date) & (df_text['date_converted'] >= min_date)]

    df_text.drop(['date_converted'], axis=1, inplace=True)
    df_text.reset_index(drop=True, inplace=True)
    df_text['group'] = df_name
    dfs_text = pd.concat([dfs_text, df_text], ignore_index=True, axis=0)
    pprint('New length for {} = {}'.format(df_name, len(df_text)))

dfs_text.drop(['index'], axis=1, inplace=True)
dfs_text.to_csv('{}{}_1_text_original.csv'.format(c.directory['save'], c.project_name), index=False)

util.display_df(dfs_text.head())
util.display_df(dfs_text.tail())

'files name = submissions'
parallel: 27 partitions with 27 cores for concat_cols_df
parallel: 27 partitions with 27 cores for str_timestamp_to_date_df
str_timestamp_to_date step is done!
'New length for submissions = 226134'
'files name = comments'
parallel: 27 partitions with 27 cores for concat_cols_df
parallel: 27 partitions with 27 cores for str_timestamp_to_date_df
str_timestamp_to_date step is done!
'New length for comments = 3333568'


,id,date,category,all_text,group
0,2qzibe,1420090180,MachineLearning,Measuring human performance on standard image ...,submissions
1,2qzppu,1420096683,dataisbeautiful,Bowls of Cereal I Ate in 2014 [OC],submissions
2,2qzslb,1420099668,dataisbeautiful,Happy New Year Message,submissions
3,2qzv35,1420102342,dataisbeautiful,European economy guide: Taking Europe’s pulse,submissions
4,2qzxp9,1420105645,dataisbeautiful,"Anchorage, Alaska never saw a day below zero i...",submissions


,id,date,category,all_text,group
3559697,ecxh5wv,1546237750,dataisbeautiful,"Ok, but the very rare exceptions don't really ...",comments
3559698,ecxhdd9,1546237967,MachineLearning,Thanks! I have a copy of that book - maybe I s...,comments
3559699,ecxhihv,1546238119,dataisbeautiful,[removed],comments
3559700,ecxhteb,1546238455,dataisbeautiful,[removed],comments
3559701,ecxi0nd,1546238689,MachineLearning,"Hi ranihorev,\n\n\nUsing the same values for n...",comments


In [9]:
### functions for cleaning ###

def preprocess_text(text):
    for replacement in c.replacements:
        try:
            text = re.sub(*replacement, text)
        except TypeError as e:
            print('{} + text {}'.format(replacement, text))
            raise e
#     return tokenizer_tag(text)
    return text

def preprocess_text_wo_tokenization_df(df):
    return df.apply(preprocess_text)

In [10]:
### cleaning using rules in the config file + saving ###

# util.init_parallel(True, c)
# dfs_text['all_text'] = dfs_text['all_text'].parallel_apply(preprocess_text)
# dfs_text['all_text'] = util.parallelize_df(dfs_text['all_text'], preprocess_text_df, c)
# dfs_text['all_text'] = util.parallelize_df_dask(dfs_text['all_text'], preprocess_text_df, c)
dfs_text['all_text'] = util.parallelize_df_dask(dfs_text['all_text'], preprocess_text_wo_tokenization_df, c)


# saving the result to file

dfs_text.to_csv('{}{}_2_text_cleaned.csv'.format(c.directory['save'], c.project_name), index=False)

pprint('size of all texts = {}'.format(dfs_text.shape))
util.display_df(dfs_text.head())

parallel: 27 partitions with 27 cores for preprocess_text_wo_tokenization_df
'size of all texts = (3559702, 5)'


,id,date,category,all_text,group
0,2qzibe,1420090180,MachineLearning,Measuring human performance on standard image ...,submissions
1,2qzppu,1420096683,dataisbeautiful,Bowls of Cereal I Ate in 2014,submissions
2,2qzslb,1420099668,dataisbeautiful,Happy New Year Message,submissions
3,2qzv35,1420102342,dataisbeautiful,European economy guide: Taking Europe’s pulse,submissions
4,2qzxp9,1420105645,dataisbeautiful,"Anchorage, Alaska never saw a day below zero i...",submissions


In [11]:
### cleaned & tagged text loading + tag file loading ###

if c.tags is not None:
    df_tags = pd.read_csv('{}{}_0_tags.csv'.format(c.directory['save'], c.project_name))
    c.tags_dict = dict(zip(df_tags['key'], df_tags['val']))

df_text_orig = pd.read_csv('{}{}_1_text_original.csv'.format(c.directory['save'], c.project_name))
df_text = pd.read_csv('{}{}_2_text_cleaned.csv'.format(c.directory['save'], c.project_name))

pprint(df_text.shape)
util.display_df(df_text.head())

(3559702, 5)


,id,date,category,all_text,group
0,2qzibe,1420090180,MachineLearning,Measuring human performance on standard image ...,submissions
1,2qzppu,1420096683,dataisbeautiful,Bowls of Cereal I Ate in 2014,submissions
2,2qzslb,1420099668,dataisbeautiful,Happy New Year Message,submissions
3,2qzv35,1420102342,dataisbeautiful,European economy guide: Taking Europe’s pulse,submissions
4,2qzxp9,1420105645,dataisbeautiful,"Anchorage, Alaska never saw a day below zero i...",submissions


In [12]:
### functions for unigram, bigram and tokenization ###

stop_words = stopwords.words('english')
stop_words.extend(c.stop_words_extensions)
nlp = spacy.load("en")
nlp.add_pipe(LanguageDetector(), name="language_detector", last=True)
ps = PorterStemmer()

def num_convertor(num):
    if num == 0:
        return 'lzerol'
    num_string = 'l'

    while num != 0:
        num_string = 'l{}{}'.format(c.num_dict[num % 10], num_string)
        num //= 10
    return num_string

def tokenizer_tag(text):
    #     print(text)
    if c.tags is None:
        return text
    txt = ' ' + text.lower() + ' '

    to_remove = string.punctuation.replace('\'', '').replace('_', '')
    table = {ord(char): ord(' ') for char in to_remove}
    txt = txt.translate(table)
    if txt.find('zzz') > -1:
        return ' '
    for ind in range(len(c.tags_dict)):
        txt = txt.replace(' ' + c.tags_dict[ind] + ' ', ' zzz' + num_convertor(ind) + 'zzz ')
    return txt

def tokenizer_tag_new(text):
    #     print(text)
    if c.tags is None:
        return text
    txt = ' ' + text.lower() + ' '

    to_remove = string.punctuation.replace('\'', '').replace('_', '')
    table = {ord(char): ord(' ') for char in to_remove}
    txt = txt.translate(table)
    
#     print(txt)
    bigrams = re.findall('\w+_\w+', txt)    
    bigrams_dict={}
    for bigram in bigrams:
        bigrams_dict[bigram.replace('_',' ')]=bigram
#     print(bigrams_dict)
    txt=txt.replace('_', ' ')
#     print(txt)

    if txt.find('zzz') > -1:
        return ' '
    for ind in range(len(c.tags_dict)):
        txt = txt.replace(' ' + c.tags_dict[ind] + ' ', ' zzz' + num_convertor(ind) + 'zzz ')

    for bigram in bigrams_dict:
        txt=txt.replace(bigram, bigrams_dict[bigram])
    
    return txt

def english_detection_spacy(text):
    doc = nlp(text)
    # document level language detection. Think of it like average language of document!
    return doc._.language['language']=='en'

def para_words_english_spacy(text):
    if c.lang_detect and not english_detection_spacy(text):
        return []
    return [ps.stem(word) for word in simple_preprocess(text, min_len=1, max_len=60, deacc=True) if word not in stop_words]

def para_words_english_spacy_df(df):
    return df.apply(para_words_english_spacy)   

def para_bigram(text):
    return bigram_mod[text]

def para_bigram_df(df):
    return df.apply(para_bigram)

def para_bigram_tokenization(text):
#     text = tokenizer_tag(' '.join(bigram_mod[text]))
    text = tokenizer_tag_new(' '.join(bigram_mod[text]))    
    return text.split()

def para_bigram_tokenization_df(df):
    return df.apply(para_bigram_tokenization)

# def make_trigrams(texts):
#     return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [13]:
### Build the bigram model ###

# util.init_parallel(True)
# words = df_text['all_text'].parallel_apply(para_words)
# words = util.parallelize_df(df_text['all_text'], para_words_df, c)
# words = util.parallelize_df_dask(df_text['all_text'], para_words_df, c)
# en_lang=english_lang(stop_words)
# words = en_lang.parallelize_english_detection_df(df_text['all_text'], c)
words = util.parallelize_df_dask(df_text['all_text'], para_words_english_spacy_df, c)

bigram = gensim.models.Phrases(words, min_count=10, threshold=10)  # higher threshold fewer phrases.
bigram_mod = gensim.models.phrases.Phraser(bigram)

# c.init_parallel(True)
# data_words_bigrams = words.parallel_apply(para_bigram)
# data_words_bigrams = util.parallelize_df(words, para_bigram_df, c)
# data_words_bigrams = util.parallelize_df_dask(words, para_bigram_df, c)
data_words_bigrams = util.parallelize_df_dask(words, para_bigram_tokenization_df, c)

# trigram = gensim.models.Phrases(bigram[words], threshold=1)
# trigram_mod = gensim.models.phrases.Phraser(trigram)

parallel: 27 partitions with 27 cores for para_words_english_spacy_df
parallel: 27 partitions with 27 cores for para_bigram_tokenization_df


In [14]:
### lemmatization ###

nlp = spacy.load('en', disable=['parser', 'ner'])
# ps = PorterStemmer()

def para_lemmatization_part(df, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    result = []

    for text in list(df['words']):
        if len(' '.join(text)) != 0:
            doc = nlp(' '.join(text))
#             result.append([ps.stem(token.lemma_.replace('_', '')) for token in doc if token.pos_ in allowed_postags])
            result.append([token.lemma_.replace('_', '') for token in doc if token.pos_ in allowed_postags])        
        else:
            result.append(list())
    df['words'] = result
    return df


df_words = pd.DataFrame(dict(words=data_words_bigrams))
# data_lemmatized = util.parallelize_df(df_words, para_lemmatization_part, c)
data_lemmatized = util.parallelize_df_dask(df_words, para_lemmatization_part, c)
data_lemmatized = data_lemmatized['words']

parallel: 27 partitions with 27 cores for para_lemmatization_part


In [15]:
### Thresholding: delete text less than the minimum threshold ###

###   7-1 min < word < % of corupus

dct = corpora.Dictionary(data_lemmatized)
dct.filter_extremes(no_below=c.no_below, no_above=c.no_above)
# corpus = [ for text in data_lemmatized]
def para_corpus(text):
    return len(dct.doc2bow(text))

def para_corpus_df(df):
    return df.apply(para_corpus)

corpus=util.parallelize_df(data_lemmatized, para_corpus_df, c)

###     7-2 min_words_per_docs

data_untagged = df_text_orig['all_text'].to_list()
count = 0
ids = []
df_temp = []
data_temp = []


count = 0
ids = []
df_temp = []
data_temp = []

for i, x in enumerate(corpus):
    if x >= c.min_keyword_threshold:
        df_temp.append(x)
        data_temp.append(data_untagged[i])
        ids.append(i)
        count += 1

data_lemmatized = data_lemmatized[ids]
data_lemmatized.reset_index(drop=True, inplace=True)
df_text = df_text.iloc[ids]
df_text.reset_index(drop=True, inplace=True)
data_untagged = data_temp
pprint('After thresholding = {}'.format(count))

parallel: 27 partitions with 27 cores for para_corpus_df
'After thresholding = 2225657'


In [16]:
### concatanating + saving ###


df_text_concat = pd.concat([
    df_text,
    pd.DataFrame(data_untagged),
    pd.DataFrame([', '.join(x) for x in data_lemmatized]),
],
    ignore_index=True,
    axis=1
)
df_cols = list(df_text_orig.columns)
df_cols.extend(['txt_orig', 'txt_lemmatized', ])
df_text_concat.columns = df_cols
df_text_concat.to_csv('{}{}_3_text_lemmatized_tagged.csv'.format(c.directory['save'], c.project_name),
                      encoding='utf-8',
                      index=False
                      )

pprint(df_text_concat.shape)
util.display_df(df_text_concat.head())

(2225657, 7)


,id,date,category,all_text,group,txt_orig,txt_lemmatized
0,2qzibe,1420090180,MachineLearning,Measuring human performance on standard image ...,submissions,Measuring human performance on standard image ...,"measur, human, zzzltwolfivelonelzzz, standard,..."
1,2qzv35,1420102342,dataisbeautiful,European economy guide: Taking Europe’s pulse,submissions,European economy guide: Taking Europe’s pulse,"european, economi, guid, take, europ, pul"
2,2qzzaa,1420107805,dataisbeautiful,Popularity of visual forms in DataIsBeautiful ...,submissions,Popularity of visual forms in DataIsBeautiful ...,"popular, zzzlthreelzerolfourlzzz, form, datais..."
3,2r048c,1420114575,dataisbeautiful,JOE GROOMING - DAILY SHAMPOO ~ best shampoo fo...,submissions,JOE GROOMING - DAILY SHAMPOO ~ best shampoo fo...,"joe, groom, daili, shampoo, good, shampoo, oil..."
4,2r0e7b,1420125946,dataisbeautiful,Los Angeles Traffic Accident Rate in Rainy vs ...,submissions,Los Angeles Traffic Accident Rate in Rainy vs ...,"trafficaccid, rate, raini, dri, weather"
